In [2]:
import os
import openai
import random
from tqdm import tqdm

# Load your API key from an environment variable or secret management service
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
import json

train_set = json.load(open('sample_train.json'))
test_set = json.load(open('sample_test.json'))
print(f'train: {len(train_set)}, test: {len(test_set)}')

train: 8400, test: 500


In [19]:
def expr2prompt(expr):
    prompt = f'Q: What is {expr}?\nA: The answer is'
    return prompt

def expr2prompt_cot(expr):
    cot_trigger = 'Let\'s think step-by-step.'
    prompt = f'Q: What is {expr}?\nA:' + ' ' + cot_trigger
    return prompt

def answer_cleansing(res):
    res = res.strip(' .')
    res = res.replace('\n', ' ').split(' ')[::-1]
    for r in res:
        try:
            r = float(r)
            return r
        except:
            pass
    return None

In [64]:
engine = "text-davinci-002"
responses = []
for sample in tqdm(test_set):
    expr, ans, e = sample
    prompt = expr2prompt(expr)
    response = openai.Completion.create(engine=engine, prompt=prompt, temperature=0, max_tokens=10)
    responses.append(response)
#     print(prompt, response["choices"][0]['text'])
#     print('GT:', ans, e)
json.dump(responses, open(f'{engine}_0_shot.json', 'w'))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [05:04<00:00,  1.64it/s]


In [83]:
eval2res = {}
for sample, response in zip(test_set, responses):
    expr, ans, e = sample
    pred = response2ans(response["choices"][0]['text'])
#     if pred is None:
#         print(expr, ans, e, response["choices"][0]['text'])
    if e not in eval2res:
        eval2res[e] = []
    if pred and int(pred) == int(eval(expr)):
        eval2res[e].append(1)
    else:
        eval2res[e].append(0)

for e, res in eval2res.items():
    print(f'{e}: {sum(res) / len(res)}')
avg_acc = sum([sum(x) for x in eval2res.values()]) / len(test_set)
print(f'Avg: {avg_acc}')

LL: 0.02
I: 0.19
LS: 0.03
SS: 0.09
SL: 0.1
Avg: 0.086


In [17]:
engine = "text-davinci-002"
direct_answer_trigger = "\nTherefore, the answer (arabic numerals) is"
responses = []
for sample in tqdm(test_set):
    expr, ans, e = sample
    x = expr2prompt_cot(expr)
    z = openai.Completion.create(engine=engine, prompt=x, temperature=0, max_tokens=1000)
    z = z["choices"][0]["text"]
    x = x + z + " " + direct_answer_trigger
    z2 = openai.Completion.create(engine=engine, prompt=x, temperature=0, max_tokens=10)
    z2 = z2["choices"][0]["text"]
    responses.append([sample, z, z2])
json.dump(responses, open(f'{engine}_0_shot_cot.json', 'w'))

100%|█████████████████████████████████████████| 500/500 [59:57<00:00,  7.19s/it]


In [20]:
eval2res = {}
for response in responses:
    expr, ans, e = response[0]
    pred = answer_cleansing(response[-1])
#     if pred is None:
#         print(expr, ans, e, response["choices"][0]['text'])
    if e not in eval2res:
        eval2res[e] = []
    if pred and int(pred) == int(eval(expr)):
        eval2res[e].append(1)
    else:
        eval2res[e].append(0)

for e, res in eval2res.items():
    print(f'{e}: {sum(res) / len(res)}')
avg_acc = sum([sum(x) for x in eval2res.values()]) / len(test_set)
print(f'Avg: {avg_acc}')

LL: 0.06
I: 0.42
LS: 0.05
SS: 0.36
SL: 0.49
Avg: 0.276
